Section 1: Imports and Data Structures

This section imports necessary libraries and defines the Process class, which serves as a blueprint for each process to be scheduled.

In [ ]:
# Import the 'sys' module to interact with the Python interpreter.
import sys
# Import 'files' from 'google.colab' to enable file uploads in Colab.
from google.colab import files
# Import 'io' to work with in-memory I/O streams for file content.
import io

# Define the Process class to hold process attributes.
class Process:
    # The constructor initializes a new Process object.
    def __init__(self, arrival_time, burst_time, pid):
        # The time the process arrives in the system.
        self.arrival_time = arrival_time
        # The total CPU time required for the process.
        self.burst_time = burst_time
        # The remaining execution time. Initially, it's the same as burst_time.
        self.remaining_time = burst_time
        # The time at which the process completes execution.
        self.completion_time = 0
        # The total time from arrival to completion.
        self.turnaround_time = 0
        # The total time the process waits in the ready queue.
        self.waiting_time = 0
        # The time from arrival until the first time the process is executed. -1 indicates it hasn't run yet.
        self.response_time = -1
        # The process ID, used for tie-breaking in some algorithms.
        self.pid = pid

Section 2: Metric Calculation

This section contains a helper function to compute the average turnaround, response, and waiting times for a list of processes after a scheduling algorithm has been run.

In [ ]:
# Function to calculate average metrics for a set of processes.
def calculate_metrics(processes):
    # Get the number of processes in the list.
    num_processes = len(processes)
    # Avoid division by zero if there are no processes.
    if num_processes == 0:
        return 0.0, 0.0, 0.0

    # Calculate the total sum of each metric.
    total_turnaround_time = sum(p.turnaround_time for p in processes)
    total_response_time = sum(p.response_time for p in processes)
    total_waiting_time = sum(p.waiting_time for p in processes)

    # Calculate the average of each metric.
    avg_turnaround_time = total_turnaround_time / num_processes
    avg_response_time = total_response_time / num_processes
    avg_waiting_time = total_waiting_time / num_processes

    # Return the average metrics as a tuple.
    return avg_turnaround_time, avg_response_time, avg_waiting_time